In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import requests, json

* read raw DM dataset from *people_list_DM.json*

In [ ]:
data_DM = pd.read_json('/content/drive/MyDrive/Accountstory/data/people_list_DM.json')
data_DM

define function that calls peopledatalabs API in order to enrich the Accountstory dataset

In [ ]:
def call_enrich_api(API_KEY, company, name):

    pdl_url = "https://api.peopledatalabs.com/v5/person/enrich"

    params = {
        "api_key": API_KEY,
        "company": [company],   
        "first_name": [name.split()[0]],
        "last_name": [name.split()[1]]
    }

    json_response = requests.get(pdl_url,  params=params).json()

    if json_response["status"] == 200:
        return json_response['data']

    else:
        print("Enrichment unsuccessful. See error and try again.")
        print("error:", json_response)

* call peopledatalabs enrichment API using employee name and employee company from DM dataset
* one at a time 

In [ ]:
key = # add PeopleDataLabs API key
N_DATAPOINTS = 400 # number of epmolyee datapoints to be enriched
start = 1000 # start index (necessary to not create duplicates when run a multiple occasions)

df = pd.DataFrame()
i = 0
for company, name in zip(data_DM[start:start+N_DATAPOINTS]['company'],data_DM[start:start+N_DATAPOINTS]['name']):
    try:
        records = call_enrich_api(key, company, name)
        df_temp = pd.DataFrame.from_dict(records, orient='index').transpose()
        if not df.empty:
            df = df.append(df_temp, ignore_index = True)
        else:
            df = df_temp
        if i%20 == 0:
            print(i)
        i += 1
    except:
        pass

In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
df.shape # (rows, features)

* Export file **people_list_DM_enriched.json**
* looks messy right now since I called the enrichment API several times and stitched already generated datasets together

In [ ]:
df_old = pd.read_json('/content/drive/MyDrive/Accountstory/data/people_list_DM_enriched_20210930.json')

In [ ]:
df = df.append(df_old)
df.shape

(1366, 71)

In [ ]:
df.reset_index(inplace=True)
df.drop('index',axis=1, inplace=True)
df

In [ ]:
df.drop_duplicates(subset=['full_name','linkedin_id','industry','job_title'], inplace=True)

In [ ]:
df.to_json('/content/drive/MyDrive/Accountstory/data/people_list_DM_enriched_20210930.json')